In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, MultiStepLR
import numpy as np
from math import *
import time

In [2]:
torch.cuda.set_device(3)

In [3]:
torch.set_default_tensor_type('torch.DoubleTensor')

In [4]:
# defination of activation function
def activation(x):
    return x * torch.sigmoid(x) 

In [5]:
# Build ResNet with three blocks
class net(torch.nn.Module):
    def __init__(self,input_width,layer_width):
        super(net,self).__init__()
        self.layer_in = torch.nn.Linear(input_width, layer_width)
        self.layer1 = torch.nn.Linear(layer_width, layer_width)
        self.layer2 = torch.nn.Linear(layer_width, layer_width)
        self.layer3 = torch.nn.Linear(layer_width, layer_width)
        self.layer4 = torch.nn.Linear(layer_width, layer_width)
        self.layer5 = torch.nn.Linear(layer_width, layer_width)
        self.layer6 = torch.nn.Linear(layer_width, layer_width)
        self.layer_out = torch.nn.Linear(layer_width, 1)
    def forward(self,x):
        y = self.layer_in(x)
        y = y + activation(self.layer2(activation(self.layer1(y)))) # residual block 1
        y = y + activation(self.layer4(activation(self.layer3(y)))) # residual block 2
        y = y + activation(self.layer6(activation(self.layer5(y)))) # residual block 3
        output = self.layer_out(y)
        return output

In [6]:
dimension = 4

In [7]:
# Defination of exact solution
def u_ex(x):  
    x_norm = torch.norm(x, dim = 1) # compute norm of x; dim = 1 - > sum by row
    u_x = torch.sin(pi/2*(1-x_norm)).reshape([x.size()[0],1])
    return u_x

In [8]:
# Defination of f(x)
def f(x):
    x_norm = torch.norm(x, dim = 1)
    u_temp = u_ex(x)
    f_temp = 1/4*pi**2*torch.sin(pi/2*(1-x_norm)) + 1/2*pi*(dimension-1)/x_norm*torch.cos(pi/2*(1-x_norm))  
    return f_temp.reshape([x.size()[0],1])+ u_temp**3

In [9]:
# Generate points by random
data_size = 1000
def generate_data():
    i = 0
    x = torch.tensor([])
    while x.size()[0] < data_size:
        temp_x = 2*torch.rand(data_size, dimension) - 1
        index = torch.where(torch.norm(temp_x, dim = 1) < 1)
        temp_x = temp_x[index]
        x = torch.cat((x,temp_x),0)
    x = x[1:data_size, :]
    return x.cuda()

In [10]:
# Defination of loss function for DRM
def loss_function(x):
    u_hat = model(x)
    ux = torch.zeros(x.size()[0], dimension).cuda()
    step_size = 0.0001
    for i in range(dimension):
        dx = torch.zeros(x.size()[0], dimension).cuda()
        dx[:, i] = torch.ones(x.size()[0])
        ux[:, i] = (model(x+step_size*dx) - model(x-step_size*dx))[:,0]/step_size/2
    uxsq = (torch.sum(ux**2, dim = 1)).reshape([x.size()[0], 1]) 
    f_temp = f(x)
    part_1 = torch.sum(1/2*uxsq + 1/4*u_hat**4 - f_temp*u_hat)/x.size()[0]
    
    return part_1

In [11]:
# Parameters initialization by normal distribution for neural networks
def initparam(model, sigma):
    for m in model.modules():
        if isinstance(m,nn.Linear):
            m.weight.data.normal_(0, sigma)
    return model

In [12]:
def model(x):
    x_norm = torch.norm(x, p = 2, dim = 1) 
    return (1-x_norm**2).reshape([x.size()[0], 1])*model1(x)

In [13]:
model1 = net(dimension, 8)
model1 = initparam(model1, 0.5)

In [14]:
device = torch.device("cuda:3" )
model1.to(device)

net(
  (layer_in): Linear(in_features=4, out_features=8, bias=True)
  (layer1): Linear(in_features=8, out_features=8, bias=True)
  (layer2): Linear(in_features=8, out_features=8, bias=True)
  (layer3): Linear(in_features=8, out_features=8, bias=True)
  (layer4): Linear(in_features=8, out_features=8, bias=True)
  (layer5): Linear(in_features=8, out_features=8, bias=True)
  (layer6): Linear(in_features=8, out_features=8, bias=True)
  (layer_out): Linear(in_features=8, out_features=1, bias=True)
)

In [15]:
# Defination of relative error
def relative_error():
    x = generate_data()
    predict = model(x)
    exact = u_ex(x)
    value = torch.sqrt(torch.sum((predict - exact)**2))/torch.sqrt(torch.sum((exact)**2))
    return value

In [16]:
traintime = 20000
error_save = np.zeros(traintime)
optimizer = optim.Adam(model1.parameters())

In [17]:
time_start = time.time()
for i in range(traintime):
    optimizer.zero_grad()
    x = generate_data()
    x.requires_grad = True
    loss = loss_function(x)
    loss.backward()
    optimizer.step()
    error = relative_error()
    error_save[i] = float(error)
    if i % 50 == 0:
        print("current epoch is: ", i)
        print("current loss is: ", loss.detach())
        print("current relative error is: ", error.detach())
        np.save("DRM_relative_error_4D_Dirichlet_nonlinear_preset_boundary.npy", error_save)
np.save("DRM_relative_error_4D_Dirichlet_nonlinear_preset_boundary.npy", error_save)
time_end = time.time()
print('total time is: ', time_end-time_start, 'seconds')

current epoch is:  0
current loss is:  tensor(0.4623, device='cuda:3')
current relative error is:  tensor(1.0083, device='cuda:3')
current epoch is:  50
current loss is:  tensor(-0.9102, device='cuda:3')
current relative error is:  tensor(0.1922, device='cuda:3')
current epoch is:  100
current loss is:  tensor(-1.0528, device='cuda:3')
current relative error is:  tensor(0.0886, device='cuda:3')
current epoch is:  150
current loss is:  tensor(-1.0500, device='cuda:3')
current relative error is:  tensor(0.0678, device='cuda:3')
current epoch is:  200
current loss is:  tensor(-1.0532, device='cuda:3')
current relative error is:  tensor(0.0465, device='cuda:3')
current epoch is:  250
current loss is:  tensor(-0.9959, device='cuda:3')
current relative error is:  tensor(0.0394, device='cuda:3')
current epoch is:  300
current loss is:  tensor(-1.0504, device='cuda:3')
current relative error is:  tensor(0.0388, device='cuda:3')
current epoch is:  350
current loss is:  tensor(-0.9437, device='c

current epoch is:  3050
current loss is:  tensor(-1.1161, device='cuda:3')
current relative error is:  tensor(0.0201, device='cuda:3')
current epoch is:  3100
current loss is:  tensor(-1.0867, device='cuda:3')
current relative error is:  tensor(0.0094, device='cuda:3')
current epoch is:  3150
current loss is:  tensor(-1.1822, device='cuda:3')
current relative error is:  tensor(0.0380, device='cuda:3')
current epoch is:  3200
current loss is:  tensor(-1.1790, device='cuda:3')
current relative error is:  tensor(0.0138, device='cuda:3')
current epoch is:  3250
current loss is:  tensor(-1.0581, device='cuda:3')
current relative error is:  tensor(0.0113, device='cuda:3')
current epoch is:  3300
current loss is:  tensor(-1.0357, device='cuda:3')
current relative error is:  tensor(0.0223, device='cuda:3')
current epoch is:  3350
current loss is:  tensor(-1.1985, device='cuda:3')
current relative error is:  tensor(0.0193, device='cuda:3')
current epoch is:  3400
current loss is:  tensor(-1.090

current epoch is:  6100
current loss is:  tensor(-1.1871, device='cuda:3')
current relative error is:  tensor(0.0175, device='cuda:3')
current epoch is:  6150
current loss is:  tensor(-1.0050, device='cuda:3')
current relative error is:  tensor(0.0464, device='cuda:3')
current epoch is:  6200
current loss is:  tensor(-1.2172, device='cuda:3')
current relative error is:  tensor(0.0578, device='cuda:3')
current epoch is:  6250
current loss is:  tensor(-1.1080, device='cuda:3')
current relative error is:  tensor(0.0263, device='cuda:3')
current epoch is:  6300
current loss is:  tensor(-1.0920, device='cuda:3')
current relative error is:  tensor(0.0166, device='cuda:3')
current epoch is:  6350
current loss is:  tensor(-1.1349, device='cuda:3')
current relative error is:  tensor(0.0151, device='cuda:3')
current epoch is:  6400
current loss is:  tensor(-1.0008, device='cuda:3')
current relative error is:  tensor(0.0215, device='cuda:3')
current epoch is:  6450
current loss is:  tensor(-1.125

current epoch is:  9150
current loss is:  tensor(-1.0013, device='cuda:3')
current relative error is:  tensor(0.0291, device='cuda:3')
current epoch is:  9200
current loss is:  tensor(-1.1345, device='cuda:3')
current relative error is:  tensor(0.0101, device='cuda:3')
current epoch is:  9250
current loss is:  tensor(-1.0128, device='cuda:3')
current relative error is:  tensor(0.0157, device='cuda:3')
current epoch is:  9300
current loss is:  tensor(-1.0614, device='cuda:3')
current relative error is:  tensor(0.0149, device='cuda:3')
current epoch is:  9350
current loss is:  tensor(-1.0711, device='cuda:3')
current relative error is:  tensor(0.0081, device='cuda:3')
current epoch is:  9400
current loss is:  tensor(-1.0365, device='cuda:3')
current relative error is:  tensor(0.0204, device='cuda:3')
current epoch is:  9450
current loss is:  tensor(-1.1597, device='cuda:3')
current relative error is:  tensor(0.0129, device='cuda:3')
current epoch is:  9500
current loss is:  tensor(-1.147

current epoch is:  12200
current loss is:  tensor(-1.0564, device='cuda:3')
current relative error is:  tensor(0.0122, device='cuda:3')
current epoch is:  12250
current loss is:  tensor(-1.0688, device='cuda:3')
current relative error is:  tensor(0.0249, device='cuda:3')
current epoch is:  12300
current loss is:  tensor(-1.0272, device='cuda:3')
current relative error is:  tensor(0.0154, device='cuda:3')
current epoch is:  12350
current loss is:  tensor(-0.9604, device='cuda:3')
current relative error is:  tensor(0.0107, device='cuda:3')
current epoch is:  12400
current loss is:  tensor(-1.0902, device='cuda:3')
current relative error is:  tensor(0.0108, device='cuda:3')
current epoch is:  12450
current loss is:  tensor(-0.9525, device='cuda:3')
current relative error is:  tensor(0.0081, device='cuda:3')
current epoch is:  12500
current loss is:  tensor(-1.0807, device='cuda:3')
current relative error is:  tensor(0.0178, device='cuda:3')
current epoch is:  12550
current loss is:  tenso

current epoch is:  15250
current loss is:  tensor(-1.1196, device='cuda:3')
current relative error is:  tensor(0.0112, device='cuda:3')
current epoch is:  15300
current loss is:  tensor(-0.9929, device='cuda:3')
current relative error is:  tensor(0.0120, device='cuda:3')
current epoch is:  15350
current loss is:  tensor(-1.1256, device='cuda:3')
current relative error is:  tensor(0.0197, device='cuda:3')
current epoch is:  15400
current loss is:  tensor(-1.0602, device='cuda:3')
current relative error is:  tensor(0.0156, device='cuda:3')
current epoch is:  15450
current loss is:  tensor(-1.2412, device='cuda:3')
current relative error is:  tensor(0.0088, device='cuda:3')
current epoch is:  15500
current loss is:  tensor(-1.0006, device='cuda:3')
current relative error is:  tensor(0.0197, device='cuda:3')
current epoch is:  15550
current loss is:  tensor(-1.0400, device='cuda:3')
current relative error is:  tensor(0.0155, device='cuda:3')
current epoch is:  15600
current loss is:  tenso

current epoch is:  18300
current loss is:  tensor(-1.1337, device='cuda:3')
current relative error is:  tensor(0.0162, device='cuda:3')
current epoch is:  18350
current loss is:  tensor(-1.0438, device='cuda:3')
current relative error is:  tensor(0.0125, device='cuda:3')
current epoch is:  18400
current loss is:  tensor(-1.0712, device='cuda:3')
current relative error is:  tensor(0.0200, device='cuda:3')
current epoch is:  18450
current loss is:  tensor(-1.1282, device='cuda:3')
current relative error is:  tensor(0.0290, device='cuda:3')
current epoch is:  18500
current loss is:  tensor(-1.0525, device='cuda:3')
current relative error is:  tensor(0.0081, device='cuda:3')
current epoch is:  18550
current loss is:  tensor(-1.0766, device='cuda:3')
current relative error is:  tensor(0.0088, device='cuda:3')
current epoch is:  18600
current loss is:  tensor(-1.1821, device='cuda:3')
current relative error is:  tensor(0.0157, device='cuda:3')
current epoch is:  18650
current loss is:  tenso